In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Flatten, Concatenate, Input, LSTM
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 뉴스 카테고리

In [ ]:
import pandas as pd

# json 읽어오기
news_data = pd.read_json('/content/drive/My Drive/News_Category_Dataset_v2.json', lines=True)
# print(news_data)

news_data = news_data.loc[:, ["category", "headline"]]
# print(news_data)

# 카테고리 정수 인코딩
# news_data['category'] = news_data['category'].replace(~~~~~~~, ~~~~~)
# print(pd.factorize(news_data['category']))
category_list = pd.factorize(news_data['category'])[1]
news_data['category'] = pd.factorize(news_data['category'])[0]

print(news_data)

In [ ]:
news_data.category.unique() # --> 41가지의 카테고리

In [ ]:
# 정규표현식 사용 --> 헤드라인 전처리, 단어 이외의 문자들은 띄어쓰기로 변경
news_data['headline'] = news_data['headline'].str.replace("[^\w]", " ")
news_data['headline']

In [ ]:
from sklearn.model_selection import train_test_split

# split하면서 shuffle 적용
news_train, news_test, y_train, y_test = train_test_split(news_data['headline'], news_data['category'], test_size=0.2, shuffle=True, random_state=23)

# 원핫벡터로 만들어줍시다! (num_classes로 카테고리 수 명시 가능)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

print(len(y_train[0]))
print(len(y_test[0]))

In [ ]:
# 토큰화 진행 --> a나 an 제거, 띄어쓰기 기준으로 문장 잘라 list에 담기 - X_train, X_test 둘 다 해줘야함!
stopwords = ['a', 'an']

X_train = []
for stc in news_train:
    token = []
    words = stc.split()
    for word in words:
        if word not in stopwords:
            token.append(word)
    X_train.append(token)

X_test = []
for stc in news_test:
    token = []
    words = stc.split()
    for word in words:
        if word not in stopwords:
            token.append(word)
    X_test.append(token)

In [ ]:
X_test

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

# 헤드라인 정수인코딩
tokenizer = Tokenizer(25000) #빈도수
tokenizer.fit_on_texts(X_train) #vocabulary 만들기
X_train = tokenizer.texts_to_sequences(X_train) # vocabulary랑 매칭해 인덱스로 나타내기
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
print(X_train)
print(X_test)

In [ ]:
print(len(tokenizer.word_index)) #  단어가 2번 이하로 나온 것들 26916개. 2번 이상 나온 것들 약 25000으로 잡고 25000 토크나이저

wc = 0
for word, word_count in tokenizer.word_counts.items():
    if word_count <= 2:
        wc += 1

print(wc)

In [ ]:
import matplotlib.pyplot as plt # 제목에 20단어가 넘는 건 거의 없음

len_stc = []
for data in X_train:
    len_stc.append(len(data))

y, x, _ = plt.hist(len_stc, bins=100)
plt.show()

In [ ]:
max_len = 15 # 헤드라인 문장 길이를 보고 맞춰주기 - 패딩
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [ ]:
model = Sequential()
model.add(Embedding(25000, 128))
model.add(LSTM(128))
model.add(Dense(41, activation='softmax')) #출력개수 41개의 카테고리

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=64, epochs=1)
#batch_size : 한번의 batch마다 주는 데이터 샘플 사이즈/ 한 번에 모든 데이터를 넣을 수가 없음
#iteration 
# epoch : 전체 데이터 셋에 대해 한 번 학습을 완료한 상태

In [ ]:
sentence = input()
token_stc = sentence.split()
encode_stc = tokenizer.texts_to_sequences([token_stc])
pad_stc = pad_sequences(encode_stc, maxlen=15)

score = model.predict(pad_stc)
print(category_list[score.argmax()], score[0, score.argmax()]) # 확률값 반환